In [1]:
import pdftotext
import re
import pandas as pd
from portfoliotools.screener.mutual_fund_screener import MutualFundScreener
from portfoliotools.screener.stock_screener import PortfolioStrategy, ADF_Reg_Model
from openpyxl.utils.dataframe import dataframe_to_rows
from shutil import copyfile
from openpyxl import load_workbook
import os

In [2]:
fileName = 'DEC2020_AA02279107_TXN.pdf' # Save the file in same folder as this notebook and specify the file name
password = 'AO********' #password for this file. Usually it is PAN No.

In [3]:
def extractHoldingsFromCas(fileName, password, analysis = False, save=True):
    with open(fileName, "rb") as f:
        pdf = pdftotext.PDF(f, password=password)

    # Find Demat Account Details
    data = pdf[1]
    data = data[data.find("In the single name of"):]
    data = [x for x in data.split('\n') if x]
    i=0
    demat_accounts = []
    for z in data:
        if (z.find(' Demat Account') >0):
            temp = [x for x in z.split('  ') if x]
            demat_accounts.append({
                'name': data[i-1].strip(),
                'count': int(temp[-2]),
                'amount':float(''.join(temp[-1].split(',')))
            })
        i+=1 
    ##################################
    # Get demat holdings
    records = []
    for page in pdf:
        records += page.split('\n')
    allData = ','.join(records)
    dematData = allData[allData.find("DEMAT ACCOUNTS HELD WITH CDSL"):allData.find("MUTUAL FUND UNITS HELD WITH MF/RTA")]
    dpIndexes = [m.start() for m in re.finditer('DP Name :', dematData)]
    j = 1
    dmat_holdings = []
    for dpIndex in dpIndexes:
        if j == len(dpIndexes):
            data = dematData[dpIndex:]
        else:
            data = dematData[dpIndex:dpIndexes[j]]
        j+=1
        i=0
        data = [x for x in data[data.find('HOLDING STATEMENT AS ON '):].split(',') if x]
        for z in data:
            temp = [x for x in z.split(' ') if x]
            i+=1
            try:
                try:
                    amount = float(temp[-1] + data[i] + data[i+1])
                except:
                    try:
                        amount = float(temp[-1] + data[i])
                    except:
                        amount = float(temp[-1])
                try:
                    quantity = float(temp[-7])
                except:
                    quantity = 0
                if amount> 0:
                    dmat_holdings.append({
                        'isin':temp[0],
                        'quantity': quantity,
                        'price': float(temp[-2]),
                        'amount': amount
                    })
            except:
                pass
    #################################
    ######### MF Holdings ###########
    mf_holdings = []
    i=0
    data = allData
    data = [x for x in data[data.find('MUTUAL FUND UNITS HELD AS ON'):].split(',') if x]
    for z in data:
        temp = [x for x in z.split(' ') if x]
        i+=1
        try:
            price = float(temp[-2])
            isin = temp[-5]
            try:
                invested = float(temp[-1] + data[i].split(' ')[0].strip())
            except:
                invested = float(temp[-1])
            try:
                quantity = float(temp[-3])
            except:
                quantity = 0
            try:
                amount = float(data[i].split(' ')[-1].strip() + data[i+1])
            except:
                try:
                    amount = float(temp[-1] + data[i].split(' ')[0].strip())
                    quantity = float(temp[-4])
                    invested = float(temp[-2])
                    price = float(temp[-3])
                    isin = temp[-6]
                except:
                    amount = float(temp[-1])
                    quantity = float(temp[-4])
                    invested = float(temp[-2])
                    price = float(temp[-3])
                    isin = temp[-6]

            if amount> 0:
                mf_holdings.append({
                    'isin':isin,
                    'quantity': quantity,
                    'price': price,
                    'invested': invested,
                    'amount': amount
                })
        except:
            pass
    holdings = dmat_holdings + mf_holdings
    holdings = pd.DataFrame(holdings).fillna(0)
    holdings['asset'] = holdings[['isin']].apply(lambda x: 'Stock' if x['isin'][:3] == 'INE' else 'MF', axis = 1)
    
    # Mutual Funds Analysis
    obj = MutualFundScreener()
    schemes = obj.searchSchemes(holdings[holdings['asset'] == 'MF']['isin'].values)
    schemes.reset_index(drop = True, inplace=True)
    schemes.rename(columns = {'scheme_isin1': 'isin'}, inplace=True)
    holdings = holdings.merge(schemes[['scheme_category', 'isin','scheme_name']], how = 'left', left_on='isin', right_on='isin', copy=False)
    holdings.fillna('',inplace=True)
    holdings['category'] = holdings[['scheme_category']].apply(lambda x: x['scheme_category'].split(' - ')[0] if x['scheme_category'] else 'Equity Scheme', axis = 1)
    holdings['scheme_category'] = holdings['scheme_category'].apply(lambda x: x if x else 'Stock')

    if analysis:
        strat = PortfolioStrategy(holdings[holdings['asset'] == 'MF']['isin'].values, period = 500)
        stats = strat.calcStat(format_result = True).T
        stats.reset_index(inplace= True)
        stats.rename(columns={'index': 'isin'}, inplace=True)
        holdings = holdings.merge(stats, how='left', left_on='isin', right_on='isin', copy=False)
        
    if save:
        template_file = 'CAS_REPORT_TEMPLATE.xlsx' 
        base_dir = 'Reports/'
        if not os.path.exists(base_dir): os.mkdir(base_dir)
        output_file = str(base_dir+fileName[:7]) + '.xlsx' 
        copyfile(template_file, output_file)
        
        # Load the workbook and access the sheet we'll paste into
        wb = load_workbook(output_file)
        writer = pd.ExcelWriter(output_file, engine='openpyxl') 
        writer.book = wb
        writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)
        holdings.to_excel(writer, "data")

        writer.save()
        
    return holdings

In [4]:
holdings = extractHoldingsFromCas(fileName, password, True, True)